# Spotify Data Case Study
I listen to a lot of music on Spotify, as do many other people. But this is about me. My problem is that I promised a friend a list of my top 50 albums and I seriously don't know what they are. I can list maybe 25 before it becomes really hard to decide. The aim of this case study is to solve my pressing issue using supervised learning. And I guess other people can also download their Spotify data do the same thing. 

I decided on using a neural network since my music taste is quite sophistocated and therefore contains many complex patterns. Specifically, it will address the problem of varying feature length better than other methods (since I listen to some albums much more than others). For this case study I won't be working with the Spotify API or any song metadata, nor will I reference album, artist, or track ID as features in the learning. 

The results were better than I expected. The neural network and logistic regression each performed noticeably better than random assignment. The measurement is admittedly a little subjective: I counted how many of their 25 album picks I would even consider for my top 50. 

**Overview**
Step 1: Contact spotify support to download my full listening history. You can only download the last year from the account page, but I want all of it. 
Step 2: Clean the data, do some EDA, fit a neural network, predict my favorite albums.

## Initialize and Clean

In [1]:
import json
import pandas as pd
import numpy as np
import tensorflow as tf
import seaborn as sns
import time
import datetime

In [2]:
# read json files to df
for i in range(0,5):
    if i == 0:
        df = pd.read_json('MyData/endsong_' + str(i) + '.json')
    else:
        # ignore_index makes sure they fully combine
        df = df.append(pd.read_json('MyData/endsong_' + str(i) + '.json'), ignore_index=True)
y_train = pd.read_csv('y_train.csv')

**Clean**

In [3]:
%%capture
# drop podcasts
df.drop(df[~df['episode_show_name'].isnull()].index) 
# drop useless columns
renames = {"master_metadata_track_name":"track","master_metadata_album_artist_name":"artist","master_metadata_album_album_name":"album"}
drops = ["username","conn_country","ip_addr_decrypted","city","region","metro_code","longitude","latitude","offline","offline_timestamp","incognito_mode","user_agent_decrypted","platform","episode_name","episode_show_name","spotify_episode_uri","spotify_track_uri"]
df.rename(columns = renames, inplace = True)
df = df.drop(drops, axis = 1)
# drop Sleepy John
df = df.drop(df[df['artist'].str.match("Sleepy John",case=False,na=False)].index)
# drop empty observations
df.drop(df[df['spotify_track_uri'].isnull()].index)
# if I do not have over 20 observations of an album I assume it can't be one of my favorites
df = df.groupby(['artist', 'album']).filter(lambda x: len(x) > 20).reset_index()
# merge df with y_train file
df = df.merge(y_train, how='left', on=['artist', 'album'])

In [4]:
# create a date column
df['date'] = df['ts'].str.split("T", expand=True)[0]
# create a timestamp for number of days since unix epoch 
# technically it's the number of seconds as a time delta, but this makes no difference 
df['date_ts'] = (pd.to_datetime(df['date']) - np.datetime64('1970-01-01T00:00:00'))

In [39]:
# here's what the data looks like
df.head(6)

,index,ts,ms_played,track,artist,album,spotify_track_uri,reason_start,reason_end,shuffle,skipped,predict,date,date_ts
0,0,2018-07-15T17:15:45Z,153351,4th Dimension,KIDS SEE GHOSTS,KIDS SEE GHOSTS,spotify:track:6JyEh4kl9DLwmSAoNDRn5b,trackdone,trackdone,False,NaN,NaN,2018-07-15,17727 days
1,4,2019-07-25T23:58:38Z,44912,I'm in Love Again,Tomppabeats,Harbor,spotify:track:3Pm3R9cbWkanONrubREjW9,trackdone,trackdone,False,NaN,NaN,2019-07-25,18102 days
2,7,2018-11-08T18:54:12Z,46359,Shimmy,System Of A Down,Toxicity,spotify:track:1a3X8Y882vwSnlnHqf9ztF,trackdone,endplay,False,NaN,NaN,2018-11-08,17843 days
3,8,2019-02-15T06:12:55Z,245098,Kids See Ghosts,KIDS SEE GHOSTS,KIDS SEE GHOSTS,spotify:track:2I3dW2dCBZAJGj5X21E53k,trackdone,trackdone,False,NaN,NaN,2019-02-15,17942 days
4,9,2017-04-17T15:17:46Z,8904,"Sing About Me, I'm Dying Of Thirst",Kendrick Lamar,"good kid, m.A.A.d city",spotify:track:0sd6BRTa0O96tfEbFGhJF9,clickrow,endplay,False,NaN,1.0,2017-04-17,17273 days
5,11,2017-03-24T19:19:26Z,493400,Holiday / Boulevard of Broken Dreams,Green Day,American Idiot,spotify:track:0MsrWnxQZxPAcov7c74sSo,trackdone,trackdone,False,NaN,NaN,2017-03-24,17249 days


**Preview**

In [40]:
%%capture
# describe all features
df.describe(include="all")

In [29]:
%%capture
# see what the frequency of trackdone and trackstart messages are
display(df["reason_end"].value_counts())
display(df["reason_start"].value_counts())

In [28]:
%%capture
# Observe the album I have listened to most: Blonde
album_grp = df.groupby(['album'])
album_grp.get_group('Blonde').head(4)

In [26]:
# 20 albums that I have the most observations for
df.groupby(['artist', 'album']).size().sort_values(ascending=False).head(20)

artist                 album                 
Frank Ocean            Blonde                    498
Chon                   Homey                     442
Flying Lotus           You're Dead!              388
Radiohead              Kid A                     324
Porter Robinson        Worlds                    317
Nujabes                Modal Soul                315
Jon Bellion            The Human Condition       301
Kanye West             The Life Of Pablo         299
Chon                   Grow                      282
Tokyo Police Club      A Lesson In Crime         270
Taylor Swift           Lover                     258
BROCKHAMPTON           SATURATION II             258
Red Hot Chili Peppers  Stadium Arcadium          254
Taylor Swift           1989                      253
Sufjan Stevens         Carrie & Lowell           244
Red Hot Chili Peppers  Californication           243
                       By the Way                242
Kendrick Lamar         good kid, m.A.A.d city    229


In [27]:
%%capture
# string matching for artist
df[df['artist'].str.match("ecco",case=False,na=False)].head(3)

## Feature and Neural Net Construction

In [12]:
# group by artist-album pair; each group will be one observation
dfgrouped = df.groupby(['artist','album'])
n_obs_max = dfgrouped.size().max()
n_groups = dfgrouped.ngroups

A = []
Y = []
# construct features, zero padding to deal with input length variation.
for name, group in dfgrouped:
    # timestamp (in days)
    a = np.array(group['date_ts'])
    a.resize(n_obs_max)
    # length of play
    b = np.array(group['ms_played'])
    b.resize(n_obs_max)
    # whether shuffle was on
    c = np.array(group['shuffle'])
    c.resize(n_obs_max)
    # standard deviation of song played. 
    # this feature distinguishes between spamming a single song and listening to many songs on the album
    e = np.array(pd.Series(group.track, dtype='category').index).std()
    A.append(a + b)
    Y.append(group['predict'].max())

A = np.vstack(A).T
A = A.astype(np.int64)
# training X
X_train = A[:,(~np.isnan(Y)).T].T
# training Y
Y = np.array(Y)[~np.isnan(Y)]

In [13]:
# the structure of this NN is extremely arbitrary. It's more of a proof of concept. 
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(128, activation='tanh'),
    tf.keras.layers.Dense(64, activation='tanh'),
    tf.keras.layers.Dense(64, activation='tanh', kernel_regularizer=tf.keras.regularizers.L2(0.1)),
    tf.keras.layers.Dense(64, activation='tanh'),
    tf.keras.layers.Dense(1, activation='sigmoid'),
])
# binary crossentropy is fine for sigmoid activation
loss_fn = tf.keras.losses.BinaryCrossentropy()

# run model, didn't seem to get much after 20 epochs
model.compile(optimizer='adam', loss=loss_fn, metrics=['accuracy'])
model.fit(X_train, Y, epochs=20)

# statistics about the prediction
# pd.DataFrame(model.predict(A.T).T[0]).describe()

Epoch 1/20
2/2 [==============================] - 0s 2ms/step - loss: 7.1178 - accuracy: 0.4872
Epoch 2/20
2/2 [==============================] - 0s 2ms/step - loss: 6.8657 - accuracy: 0.6923
Epoch 3/20
2/2 [==============================] - 0s 3ms/step - loss: 6.6559 - accuracy: 0.7179
Epoch 4/20
2/2 [==============================] - 0s 3ms/step - loss: 6.4610 - accuracy: 0.6923
Epoch 5/20
2/2 [==============================] - 0s 2ms/step - loss: 6.2782 - accuracy: 0.7436
Epoch 6/20
2/2 [==============================] - 0s 2ms/step - loss: 6.1036 - accuracy: 0.7436
Epoch 7/20
2/2 [==============================] - 0s 3ms/step - loss: 5.9354 - accuracy: 0.7949
Epoch 8/20
2/2 [==============================] - 0s 2ms/step - loss: 5.7729 - accuracy: 0.8205
Epoch 9/20
2/2 [==============================] - 0s 3ms/step - loss: 5.6119 - accuracy: 0.8205
Epoch 10/20
2/2 [==============================] - 0s 2ms/step - loss: 5.4522 - accuracy: 0.8718
Epoch 11/20
2/2 [======================

In [14]:
# create grouped df to add predicted Y values. Inappropriately named 'test_y'
dfgrouped_merge = pd.DataFrame(dfgrouped.size())
dfgrouped_merge.columns = ['test_y']
dfgrouped_merge['test_y'] = model.predict(A.T).T[0]
dfgrouped_merge = df.merge(dfgrouped_merge, how='left', on=['artist', 'album'])

In [15]:
#df[df.test_y == 1].groupby(['artist','album','test_y']).size().head(55)
def f(x):
    k = f['test_y'].mean()
    return pd.Series(k, index='test_y')

df_predictions = pd.DataFrame(dfgrouped_merge.groupby(['artist','album']).apply(
    lambda x: pd.Series([x.test_y.mean(),x.predict.mean()], index=['test_y','predict']))).sort_values('test_y',ascending=0)

## Neural Net Results
Here I ran through a couple different ideas. Theres some variation due to the random starting parameters with gradient descent, so it's good to test out a couple things. I decided scrambling the predicted values was a good way to check the neural net against randomly chosen songs. It's a start at seeing how overfit the model is. I was also curious how a simple logistic regression would perform versus the "deep" neural net. Honestly the 'logistic regression' (single feature sigmoid) works pretty well, and it seems to catch a slightly different trend in the data, pulling up a couple good picks that the neural net missed. 

In [16]:
df_predictions[df_predictions.predict!=1].head(25)
# 1. ) NN set: 12/25 are considerations in my mind; Random set: 9/50 I barely recognized ~20 of them
# 2. ) ADDED "SONG SPAMMING FEATURE":    NN set: 10/25; Random set: 11/47  not sure what to make of the difference if any
# 3. )^ again:    NN set: 19/25. There are albums it consistently likes, that I like too. 
# The top 5-10 are consistently some of my considerations
# Random set: 10/44; Seems worse than the NN, but its pretty helpful in its own way. Randomly listing albums jogs my memory.
# 4. ) "logistic" regression (1 feature sigmoid NN). Performs exceptional on train set. 14/25 on test set !!!
# Random Set: 7/47; again some good picks that don't show up on the NN or LR.
# 5. ) Logistic regression again. 7/25   Random assignment: 10/45
# 6. ) Logistic 13/25, NN: 12/25, Random: not much different from other times
# 7. ) Added much more training data. 

,,test_y,predict
artist,album,,
Tomppabeats,Harbor,0.924204,NaN
BROCKHAMPTON,GINGER,0.896664,NaN
Kendrick Lamar,To Pimp A Butterfly,0.896664,NaN
Kanye West,The Life Of Pablo,0.890843,NaN
Madvillain,Madvillainy,0.880098,NaN
Rae Sremmurd,SremmLife 2,0.869153,NaN
XXXTENTACION,17,0.869153,NaN
"Tyler, The Creator",IGOR,0.866420,NaN
Charly Bliss,Guppy,0.861609,NaN


## Random Assignment Results

In [17]:
# create a column of the same predictions, but permuted randomly
df_predictions['test_y_scramble'] = list(df_predictions.test_y.sample(frac=1).reset_index(drop=1))
# look at scrambled prediction values to see if the neural network is doing anything productive
# so far looks more effective than randomness... but there's a good chance of confirmation bias
df_predictions.sort_values('test_y_scramble',ascending=0).head(50)

,,test_y,predict,test_y_scramble
artist,album,,,
Mac DeMarco,This Old Dog,0.331755,NaN,0.964504
Jimi Hendrix,Electric Ladyland,0.320928,NaN,0.958369
Ugly Casanova,Sharpen Your Teeth,0.103632,NaN,0.957338
System Of A Down,Toxicity,0.680597,NaN,0.943285
Philanthrope,Clockwork,0.017902,NaN,0.931907
KOAN Sound,Forgotten Myths,0.408924,NaN,0.924204
Taylor Swift,Fearless,0.506960,NaN,0.914800
DJ Okawari,Libyus Music Sound History 2004-2010,0.242424,NaN,0.896664
Drake,More Life,0.196324,0.0,0.896664


In [35]:
%%capture
# plot data to look for what features correlate with the model's predictions
df2 = df.merge(df_predictions, on=['artist','album'])
sns.scatterplot(data=df2, x='shuffle', y='test_y')

## Logistic Regression Results

In [36]:
# lets try a logistic regression. Seems to work as well as the NN


model_logitstic_regression = tf.keras.models.Sequential([
    tf.keras.layers.Dense(1, activation='sigmoid'),
])
loss_fn = tf.keras.losses.BinaryCrossentropy()
model_logitstic_regression.compile(optimizer='adam', loss=loss_fn, metrics=['accuracy'])

model_logitstic_regression.fit(X_train, Y, epochs=200)

# statistics about the prediction
# pd.DataFrame(model.predict(A.T).T[0]).describe()

dfgrouped_merge_LR = pd.DataFrame(dfgrouped.size())
dfgrouped_merge_LR.columns = ['test_y']
dfgrouped_merge_LR['test_y'] = model_logitstic_regression.predict(A.T).T[0]
dfgrouped_merge_LR = df.merge(dfgrouped_merge_LR, how='left', on=['artist', 'album'])

#df[df.test_y == 1].groupby(['artist','album','test_y']).size().head(55)
def f(x):
    k = f['test_y'].mean()
    return pd.Series(k, index='test_y')

df_predictions_LR = pd.DataFrame(dfgrouped_merge.groupby(['artist','album']).apply(
    lambda x: pd.Series([x.test_y.mean(),x.predict.mean()], index=['test_y','predict']))).sort_values('test_y',ascending=0)

df_predictions_LR[df_predictions_LR.predict!=1].head(25)


# create a column of the same predictions, but permuted randomly
df_predictions_LR['test_y_scramble'] = list(df_predictions_LR.test_y.sample(frac=1).reset_index(drop=1))
# look at scrambled prediction values to see if the neural network is doing anything productive
# so far looks more effective than randomness... but there's a good chance of confirmation bias


Epoch 1/200
2/2 [==============================] - 0s 2ms/step - loss: 977034347498438656.0000 - accuracy: 0.4872
Epoch 2/200
2/2 [==============================] - 0s 2ms/step - loss: 684779346472206336.0000 - accuracy: 0.4359
Epoch 3/200
2/2 [==============================] - 0s 2ms/step - loss: 416611243596447744.0000 - accuracy: 0.4359
Epoch 4/200
2/2 [==============================] - 0s 2ms/step - loss: 198958037795340288.0000 - accuracy: 0.4103
Epoch 5/200
2/2 [==============================] - 0s 1ms/step - loss: 169972059428880384.0000 - accuracy: 0.4103
Epoch 6/200
2/2 [==============================] - 0s 1ms/step - loss: 226788206102708224.0000 - accuracy: 0.4359
Epoch 7/200
2/2 [==============================] - 0s 2ms/step - loss: 242602000808673280.0000 - accuracy: 0.4103
Epoch 8/200
2/2 [==============================] - 0s 1ms/step - loss: 208888826817413120.0000 - accuracy: 0.4103
Epoch 9/200
2/2 [==============================] - 0s 1ms/step - loss: 14969385237715353

2/2 [==============================] - 0s 2ms/step - loss: 3343160398839808.0000 - accuracy: 0.8462
Epoch 73/200
2/2 [==============================] - 0s 2ms/step - loss: 3388723827834880.0000 - accuracy: 0.8718
Epoch 74/200
2/2 [==============================] - 0s 2ms/step - loss: 3201148378939392.0000 - accuracy: 0.8718
Epoch 75/200
2/2 [==============================] - 0s 999us/step - loss: 3738576026075136.0000 - accuracy: 0.8718
Epoch 76/200
2/2 [==============================] - 0s 998us/step - loss: 3226589617717248.0000 - accuracy: 0.8974
Epoch 77/200
2/2 [==============================] - 0s 2ms/step - loss: 4073194512187392.0000 - accuracy: 0.8462
Epoch 78/200
2/2 [==============================] - 0s 999us/step - loss: 3071600354131968.0000 - accuracy: 0.8974
Epoch 79/200
2/2 [==============================] - 0s 2ms/step - loss: 2675745566490624.0000 - accuracy: 0.9231
Epoch 80/200
2/2 [==============================] - 0s 1000us/step - loss: 3202792009236480.0000 - accu

2/2 [==============================] - 0s 1ms/step - loss: 1123466539433984.0000 - accuracy: 0.9487
Epoch 145/200
2/2 [==============================] - 0s 2ms/step - loss: 1371263738052608.0000 - accuracy: 0.9487
Epoch 146/200
2/2 [==============================] - 0s 2ms/step - loss: 1098574251163648.0000 - accuracy: 0.9487
Epoch 147/200
2/2 [==============================] - 0s 2ms/step - loss: 1153042355322880.0000 - accuracy: 0.9487
Epoch 148/200
2/2 [==============================] - 0s 2ms/step - loss: 1175826619957248.0000 - accuracy: 0.8974
Epoch 149/200
2/2 [==============================] - 0s 2ms/step - loss: 1117621827141632.0000 - accuracy: 0.9231
Epoch 150/200
2/2 [==============================] - 0s 2ms/step - loss: 1087827571900416.0000 - accuracy: 0.9487
Epoch 151/200
2/2 [==============================] - 0s 2ms/step - loss: 1086477811318784.0000 - accuracy: 0.9487
Epoch 152/200
2/2 [==============================] - 0s 2ms/step - loss: 1095908284432384.0000 - accur

In [38]:
df_predictions_LR[df_predictions.predict!=1].head(25)

,,test_y,predict,test_y_scramble
artist,album,,,
Tomppabeats,Harbor,0.924204,NaN,0.072582
BROCKHAMPTON,GINGER,0.896664,NaN,0.367004
Kendrick Lamar,To Pimp A Butterfly,0.896664,NaN,0.449296
Kanye West,The Life Of Pablo,0.890843,NaN,0.444893
Madvillain,Madvillainy,0.880098,NaN,0.498583
Rae Sremmurd,SremmLife 2,0.869153,NaN,0.543826
XXXTENTACION,17,0.869153,NaN,0.057449
"Tyler, The Creator",IGOR,0.866420,NaN,0.012642
Charly Bliss,Guppy,0.861609,NaN,0.040976
